In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

In [87]:
# Import data 
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [88]:
# Create seperate dataframe of item identifier and item weight 
item = train[["Item_Identifier","Item_Weight"]]
item[item.Item_Weight.isnull()]["Item_Identifier"].nunique()
len(item)

8523

In [89]:
# Seperate dataframe of unique items and thier weights 
unique_weights = item.sort_values("Item_Identifier").drop_duplicates().dropna()
unique_weights.head()

,Item_Identifier,Item_Weight
8043,DRA12,11.600
8195,DRA24,19.350
3927,DRA59,8.270
3465,DRB01,7.390
1836,DRB13,6.115


In [90]:
item_non_missing = pd.merge(item,unique_weights,how='left',on = "Item_Identifier")

In [91]:
item_non_missing.sort_values("Item_Identifier").isnull().sum()
item_non_missing = item_non_missing[['Item_Identifier','Item_Weight_y']]
item_non_missing.dtypes

Item_Identifier     object
Item_Weight_y      float64
dtype: object

In [92]:
#pd.concat(["train","item_non_missing"],axis=1)
train.drop("Item_Identifier",axis=1)

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [93]:
training_Set = pd.concat(objs=[item_non_missing,train.drop(labels="Item_Identifier",axis=1)],axis=1)
training_Set = training_Set.drop(labels=["Item_Weight","Outlet_Identifier"],axis=1)
training_Set.head()

,Item_Identifier,Item_Weight_y,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,1987,High,Tier 3,Supermarket Type1,994.7052


In [94]:
# Missing values for item fat 
def item_fat (x):
    if x=="low fat":
        return "Low Fat"
    elif x=="LF":
        return "Low Fat"
    elif x=="reg":
        return "Regular"
    else:
        return x
    
training_Set['Item_Fat_Content'] = training_Set.Item_Fat_Content.apply(item_fat)

In [95]:
training_Set["Outlet_Size"] = training_Set.Outlet_Size.fillna("Unknown")

In [113]:
training_Set[training_Set.Item_Weight_y.isnull()]

,Item_Identifier,Item_Weight_y,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales


In [112]:
item_type =training_Set.groupby('Item_Type')
item_type.median()

# Frozen foods 12.782404
# snack foods 13.031230
# dairy 13.379905
# baking goods 12.285317

training_Set["Item_Weight_y"].iloc[927] = 12.782404
training_Set["Item_Weight_y"].iloc[1922] = 13.031230
training_Set["Item_Weight_y"].iloc[4187] = 13.379905
training_Set["Item_Weight_y"].iloc[5022] = 12.285317


/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [117]:
training_Set["Item_Weight_y"]=np.round(training_Set.Item_Weight_y,2)
training_Set.head()

,Item_Identifier,Item_Weight_y,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,1998,Unknown,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,1987,High,Tier 3,Supermarket Type1,994.7052


12.782404